In [ ]:
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update -y  
!sudo apt-get install -y google-chrome-stable
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/
!google-chrome --version
!chromedriver -v
!pip install selenium

from bs4 import BeautifulSoup
from PIL import Image
from selenium import webdriver
import requests
import os
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:

url = 'https://iprice.co.id/harga/oppo-a17/'

chrome_options = webdriver.ChromeOptions()
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
driver = webdriver.Chrome(options=chrome_options)
# driver.findElement(By.xpath("//*[contains(@id, '_default-create-firstname')]"));


driver.get(url)

status = True

while status == True:
    try:
        button = WebDriverWait(driver, 10).until(
                (EC.element_to_be_clickable((By.ID, "show-more-offers"))))
        button.click()

    except:
        status = False
        print()

status = True

while status == True:

    try:
        button = WebDriverWait(driver, 10).until(
                (EC.element_to_be_clickable((By.XPATH, "//*[contains(@id, 'iptoffersForm')]"))))
        button.click()

    except:
        status = False
        print()
        

# driver.save_screenshot('result.png')
# img = Image.open('result.png')
# img


import json
page_source = driver.page_source
soup_source = BeautifulSoup(page_source, 'lxml')
prices = []
prices_selector_span = soup_source.find_all('span', attrs={"class" : "kq ek dF lZ db bR o8 ks li mG"})

prices_selector_id = soup_source.find_all('a', attrs={"class" : "db ek dF lZ bR o8 ks li mG"})
for j in range (len(prices_selector_id)):
    price = prices_selector_id[j].get('on')
    singlequote = "'"
    doublequote = '"'
    
    try:
        obj = json.loads(price[0:price.find("),AMP.navigateTo")]
        .replace("tap:AMP.setState(","")
        .replace(singlequote,doublequote)
        .replace('offerId', '"offerId"')
        .replace('feedbackState', '"feedbackState"')
        .replace('position', '"position"')
        .replace('sale', '"sale"')
        .replace('currency', '"currency"')
        .replace('discount','"discount"')
        .replace('original',"original")
        .replace('currentShowButtonFeedbackId', '"currentShowButtonFeedbackId"')
        .replace('original','"original"')
        .replace('section','"section"')
        .replace('new-offers','new-offers'))
        
        sale = obj["feedbackState"]["sale"]
        prices.append(sale)
    
        
    except:
        print()
        
        
    
    

for i in range (len(prices_selector_span)):
    price = prices_selector_span[i].get('on')
    singlequote = "'"
    doublequote = '"'
    
    try:
        obj = json.loads(price[0:price.find("),AMP.navigateTo")]
        .replace("tap:AMP.setState(","")
        .replace(singlequote,doublequote)
        .replace('offerId', '"offerId"')
        .replace('feedbackState', '"feedbackState"')
        .replace('position', '"position"')
        .replace('sale', '"sale"')
        .replace('currency', '"currency"')
        .replace('discount','"discount"')
        .replace('original',"original")
        .replace('currentShowButtonFeedbackId', '"currentShowButtonFeedbackId"')
        .replace('original','"original"')
        .replace('section','"section"')
        .replace('new-offers','"new-offers"'))
        
        sale = obj["feedbackState"]["sale"]
        prices.append(sale)
        
    except Exception as e:

        print(e)
        
    
    

prices = pd.DataFrame(prices)
prices.to_csv("prices.csv", index=False)
driver.quit()
prices